# NLP501 – Exercise 2: Probabilistic Models & Language Modeling

**Student:** Nguyễn Đức Bình  
**Course:** NLP501 – Natural Language Processing  
**Exercise:** 2 (Autocorrect System • POS Tagging with HMM • N-gram Language Model)

This notebook implements all 3 parts required in the assignment:

1. **Autocorrect System** using **Minimum Edit Distance (Levenshtein)**
2. **POS Tagging** using **Hidden Markov Model (HMM)** + **Viterbi decoding**
3. **N-gram Language Model** (unigram/bigram/trigram) with **Add-k smoothing**, **perplexity**, and **autocomplete**

>  Everything is written to be easy to run end-to-end.


In [9]:
# =========================
# 0. Setup & Imports
# =========================
import re
import math
import random
from collections import Counter, defaultdict
from typing import List, Tuple, Dict

import numpy as np

# NLTK for corpora
import nltk

# Download required NLTK datasets (run once)
nltk.download('punkt')
nltk.download('brown')
nltk.download('treebank')
nltk.download('universal_tagset')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

## Part 1 — Autocorrect System (Edit Distance)

###  Goal

Build an autocorrect system that:

- Implements **Minimum Edit Distance (Levenshtein)**
- Builds a **vocabulary** from an English corpus
- Uses **word frequency** to rank suggestions
- Combines edit distance + frequency to pick best correction
- Supports **distance 1–2**
- Returns **top 5 suggestions**
- Handles **lowercase & uppercase**


In [10]:
# =========================
# Part 1.1 — Text processing + Vocabulary
# =========================

def tokenize_words(text: str) -> List[str]:
    """Extract words (letters + apostrophes) and lowercase them."""
    return re.findall(r"[a-zA-Z']+", text.lower())

def build_vocabulary_from_corpus_words(words: List[str], min_freq: int = 1) -> Tuple[set, Counter]:
    """Build vocabulary set and frequency counter."""
    freqs = Counter(words)
    vocab = {w for w, c in freqs.items() if c >= min_freq}
    return vocab, freqs

# Use Brown corpus as default corpus for vocab
from nltk.corpus import brown

brown_words = [w.lower() for w in brown.words()]
vocab, word_freqs = build_vocabulary_from_corpus_words(brown_words, min_freq=1)

print(f"Vocabulary size: {len(vocab):,}")
print("Top 10 words:", word_freqs.most_common(10))


Vocabulary size: 49,815
Top 10 words: [('the', 69971), (',', 58334), ('.', 49346), ('of', 36412), ('and', 28853), ('to', 26158), ('a', 23195), ('in', 21337), ('that', 10594), ('is', 10109)]


In [11]:
# =========================
# Part 1.2 — Minimum Edit Distance (Levenshtein)
# =========================

def levenshtein_distance(a: str, b: str) -> int:
    """
    Compute Minimum Edit Distance (Levenshtein):
    insert / delete / substitute all cost 1
    """
    a = a.lower()
    b = b.lower()
    n, m = len(a), len(b)
    dp = [[0]*(m+1) for _ in range(n+1)]

    for i in range(n+1):
        dp[i][0] = i
    for j in range(m+1):
        dp[0][j] = j

    for i in range(1, n+1):
        for j in range(1, m+1):
            cost = 0 if a[i-1] == b[j-1] else 1
            dp[i][j] = min(
                dp[i-1][j] + 1,      # deletion
                dp[i][j-1] + 1,      # insertion
                dp[i-1][j-1] + cost  # substitution
            )
    return dp[n][m]

# Quick tests
tests = [
    ("kitten", "sitting"),
    ("book", "back"),
    ("NLP", "nlp"),
    ("speling", "spelling"),
]
for a, b in tests:
    print(a, b, levenshtein_distance(a,b))


kitten sitting 3
book back 2
NLP nlp 0
speling spelling 1


In [12]:
# =========================
# Part 1.3 — Candidate generation (edit distance 1 & 2)
# =========================

alphabet = 'abcdefghijklmnopqrstuvwxyz'

def edits1(word: str) -> set:
    """All edits that are one edit away from `word`."""
    word = word.lower()
    splits = [(word[:i], word[i:]) for i in range(len(word)+1)]

    deletes    = [L + R[1:]           for L, R in splits if R]
    inserts    = [L + c + R           for L, R in splits for c in alphabet]
    replaces   = [L + c + R[1:]       for L, R in splits if R for c in alphabet]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]

    return set(deletes + inserts + replaces + transposes)

def edits2(word: str) -> set:
    """All edits that are two edits away from `word`."""
    return set(e2 for e1 in edits1(word) for e2 in edits1(e1))

def known(words: set, vocab: set) -> set:
    """Filter candidates that exist in vocabulary."""
    return set(w for w in words if w in vocab)

def get_candidates(word: str, vocab: set, max_distance: int = 2) -> Dict[int, set]:
    """Return candidate words grouped by edit distance."""
    word_l = word.lower()
    out = defaultdict(set)

    if word_l in vocab:
        out[0].add(word_l)
        return out

    c1 = known(edits1(word_l), vocab)
    out[1] = c1

    if max_distance >= 2:
        c2 = known(edits2(word_l), vocab)
        # Avoid duplicates already in distance 1
        out[2] = c2 - c1

    return out

# Demo candidates
word = "speling"
cands = get_candidates(word, vocab, max_distance=2)
for d in sorted(cands.keys()):
    print(d, len(cands[d]), list(sorted(list(cands[d])))[:10])


1 2 ['spelling', 'spewing']
2 40 ['dueling', 'feeling', 'opening', 'paling', 'peeling', 'peking', 'pelting', 'piling', 'poling', 'reeling']


In [13]:
# =========================
# Part 1.4 — Ranking suggestions (edit distance + frequency)
# =========================

def suggest_corrections(word: str, vocab: set, freqs: Counter, top_k: int = 5, max_distance: int = 2):
    """
    Return top_k suggestions sorted by:
      1) smaller edit distance
      2) higher frequency
    Keeps original casing for output.
    """
    original = word
    w = word.lower()

    candidates_by_d = get_candidates(w, vocab, max_distance=max_distance)

    ranked = []
    for d in sorted(candidates_by_d.keys()):
        for cand in candidates_by_d[d]:
            ranked.append((d, freqs[cand], cand))

    ranked.sort(key=lambda x: (x[0], -x[1], x[2]))  # dist asc, freq desc
    ranked = ranked[:top_k]

    # Restore casing: if original is capitalized, capitalize suggestions
    if original[:1].isupper():
        return [(d, f, c.capitalize()) for d, f, c in ranked]
    return ranked

# Test suggestions
for w in ["speling", "acress", "Langauge", "thier", "wrold"]:
    print("\nInput:", w)
    print(suggest_corrections(w, vocab, word_freqs, top_k=5, max_distance=2))



Input: speling
[(1, 4, 'spelling'), (1, 1, 'spewing'), (2, 172, 'feeling'), (2, 127, 'spring'), (2, 86, 'seeing')]

Input: acress
[(1, 282, 'across'), (1, 44, 'acres'), (1, 24, 'access'), (1, 6, 'actress'), (1, 1, 'caress')]

Input: Langauge
[(1, 109, 'Language'), (2, 40, 'Languages')]

Input: thier
[(1, 2669, 'their'), (1, 8, 'thief'), (1, 5, 'ther'), (2, 69971, 'the'), (2, 5145, 'this')]

Input: wrold
[(1, 787, 'world'), (1, 1, 'wold'), (2, 2714, 'would'), (2, 661, 'old'), (2, 413, 'told')]


In [14]:
# =========================
# Part 1.5 — Interactive CLI demo
# =========================

def autocorrect_sentence(sentence: str, vocab: set, freqs: Counter, max_distance: int = 2, top_k: int = 5):
    """Autocorrect each word (simple token-based)."""
    tokens = re.findall(r"[A-Za-z']+|[^A-Za-z']+", sentence)
    corrected = []
    for tok in tokens:
        if re.fullmatch(r"[A-Za-z']+", tok):
            if tok.lower() in vocab:
                corrected.append(tok)
            else:
                sug = suggest_corrections(tok, vocab, freqs, top_k=top_k, max_distance=max_distance)
                corrected.append(sug[0][2] if sug else tok)
        else:
            corrected.append(tok)
    return ''.join(corrected)

def interactive_autocorrect():
    print("\n=== Autocorrect Demo (type 'exit' to stop) ===")
    while True:
        s = input("Enter a sentence: ")
        if s.strip().lower() == 'exit':
            break
        print("Corrected:", autocorrect_sentence(s, vocab, word_freqs))
        print("\nSuggestions per word:")
        for w in re.findall(r"[A-Za-z']+", s):
            if w.lower() not in vocab:
                print(f"  {w} -> {suggest_corrections(w, vocab, word_freqs)}")

# Uncomment to run interactively:
# interactive_autocorrect()

print("Ready. To run demo: uncomment interactive_autocorrect()")


Ready. To run demo: uncomment interactive_autocorrect()


## Part 2 — POS Tagging with HMM

###  Goal

Build a POS tagger using:

- Transition probabilities: \(P(tag*i \mid tag*{i-1})\)
- Emission probabilities: \(P(word \mid tag)\)
- **Viterbi algorithm** for decoding
- Handle **unknown words** with smoothing
- Evaluate **accuracy** on a test set

Dataset requirement in the exercise:

- WSJ corpus (NLTK Treebank) **or** Brown corpus with simplified tags.

I'll use **Treebank + Universal Tagset** for a standard POS tagging task.


In [15]:
# =========================
# Part 2.1 — Load dataset (Treebank / WSJ) and split train/test
# =========================
from nltk.corpus import treebank

# Tagged sentences from WSJ Treebank
tagged_sents = list(treebank.tagged_sents(tagset='universal')) # Convert to list

# Shuffle and split
random.seed(42)
random.shuffle(tagged_sents)

split = int(0.8 * len(tagged_sents))
train_sents = tagged_sents[:split]
test_sents  = tagged_sents[split:]

print(f"Total sentences: {len(tagged_sents)}")
print(f"Train: {len(train_sents)} | Test: {len(test_sents)}")
print("Example:", train_sents[0][:10])

Total sentences: 3914
Train: 3131 | Test: 783
Example: [('The', 'DET'), ('company', 'NOUN'), ('noted', 'VERB'), ('that', 'ADP'), ('it', 'PRON'), ('has', 'VERB'), ('reduced', 'VERB'), ('debt', 'NOUN'), ('by', 'ADP'), ('$', '.')]


In [16]:
# =========================
# Part 2.2 — Train HMM parameters (pi, transition, emission) with smoothing
# =========================

START = "<s>"
END = "</s>"
UNK = "<UNK>"

def build_hmm_counts(tagged_sentences):
    tag_counts = Counter()
    word_counts = Counter()
    transition_counts = Counter()
    emission_counts = Counter()
    start_counts = Counter()

    for sent in tagged_sentences:
        prev_tag = START
        start_counts[sent[0][1]] += 1

        for word, tag in sent:
            w = word.lower()
            tag_counts[tag] += 1
            word_counts[w] += 1
            transition_counts[(prev_tag, tag)] += 1
            emission_counts[(tag, w)] += 1
            prev_tag = tag

        transition_counts[(prev_tag, END)] += 1

    tags = sorted(tag_counts.keys())
    vocab_words = set(word_counts.keys())
    return tags, vocab_words, tag_counts, transition_counts, emission_counts, start_counts

tags, train_vocab_words, tag_counts, trans_counts, emit_counts, start_counts = build_hmm_counts(train_sents)

print(f"#Tags: {len(tags)}")
print("Tags:", tags)


#Tags: 12
Tags: ['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


In [17]:
# =========================
# Part 2.3 — Convert counts to probabilities (log-space)
# Using Laplace smoothing for transitions & emissions
# =========================

def train_hmm_params(tags, vocab_words, tag_counts, trans_counts, emit_counts, start_counts,
                     alpha_trans: float = 1.0, alpha_emit: float = 1.0):
    """
    Return log-prob dictionaries for:
      - log_pi[tag]
      - log_A[prev_tag][tag]
      - log_B[tag][word]
    Includes UNK in emission vocab.
    """
    vocab = set(vocab_words) | {UNK}
    num_tags = len(tags)
    V = len(vocab)

    # Initial probabilities pi
    total_starts = sum(start_counts.values())
    log_pi = {}
    for t in tags:
        # smoothing for unseen start tags
        log_pi[t] = math.log((start_counts[t] + alpha_trans) / (total_starts + alpha_trans * num_tags))

    # Transition probabilities A
    log_A = {pt: {} for pt in [START] + tags}
    for prev_t in [START] + tags:
        # total outgoing transitions from prev_t
        total_out = 0
        for t in tags + [END]:
            total_out += trans_counts[(prev_t, t)]
        denom = total_out + alpha_trans * (len(tags) + 1)  # +END

        for t in tags:
            log_A[prev_t][t] = math.log((trans_counts[(prev_t, t)] + alpha_trans) / denom)
        # END transition separately
        log_A[prev_t][END] = math.log((trans_counts[(prev_t, END)] + alpha_trans) / denom)

    # Emission probabilities B
    log_B = {t: {} for t in tags}
    for t in tags:
        denom = tag_counts[t] + alpha_emit * V
        for w in vocab:
            log_B[t][w] = math.log((emit_counts[(t, w)] + alpha_emit) / denom)

    return vocab, log_pi, log_A, log_B

vocab_hmm, log_pi, log_A, log_B = train_hmm_params(
    tags, train_vocab_words, tag_counts, trans_counts, emit_counts, start_counts,
    alpha_trans=1.0, alpha_emit=1.0
)

print("HMM parameters trained.")
print("Example log_pi for top tags:")
for t in tags[:5]:
    print(t, log_pi[t])


HMM parameters trained.
Example log_pi for top tags:
. -2.480779004619803
ADJ -3.185398586341985
ADP -2.036775877099214
ADV -2.9530666089733693
CONJ -2.9715286718131044


In [18]:
# =========================
# Part 2.4 — Viterbi decoding
# =========================

def viterbi_decode(words: List[str], tags: List[str], vocab: set,
                   log_pi: Dict[str, float],
                   log_A: Dict[str, Dict[str, float]],
                   log_B: Dict[str, Dict[str, float]]):
    """
    Return best tag sequence for input words using Viterbi (log probabilities).
    """
    # Convert to lowercase, map unknown to UNK
    obs = [w.lower() if w.lower() in vocab else UNK for w in words]
    n = len(obs)

    # dp[t][i] = best log prob ending in tag t at position i
    dp = {t: [-float('inf')] * n for t in tags}
    back = {t: [None] * n for t in tags}

    # init
    for t in tags:
        dp[t][0] = log_pi[t] + log_B[t][obs[0]]
        back[t][0] = START

    # recursion
    for i in range(1, n):
        for t in tags:
            best_prev = None
            best_score = -float('inf')
            for pt in tags:
                score = dp[pt][i-1] + log_A[pt][t] + log_B[t][obs[i]]
                if score > best_score:
                    best_score = score
                    best_prev = pt
            dp[t][i] = best_score
            back[t][i] = best_prev

    # termination to END
    best_last_tag = None
    best_last_score = -float('inf')
    for t in tags:
        score = dp[t][n-1] + log_A[t][END]
        if score > best_last_score:
            best_last_score = score
            best_last_tag = t

    # backtrack
    best_tags = [best_last_tag]
    for i in range(n-1, 0, -1):
        best_tags.append(back[best_tags[-1]][i])
    best_tags.reverse()
    return best_tags

# Quick demo
sample_words = [w for w,_ in test_sents[0]]
pred_tags = viterbi_decode(sample_words, tags, vocab_hmm, log_pi, log_A, log_B)
print("Sentence:", sample_words[:12])
print("Pred:", pred_tags[:12])
print("Gold:", [t for _,t in test_sents[0]][:12])


Sentence: ['Terms', 'were', "n't", 'disclosed', '*-1', '.']
Pred: ['NOUN', 'VERB', 'ADV', 'VERB', 'X', '.']
Gold: ['NOUN', 'VERB', 'ADV', 'VERB', 'X', '.']


In [19]:
# =========================
# Part 2.5 — Evaluate accuracy
# =========================

def evaluate_hmm(test_tagged_sents):
    total = 0
    correct = 0

    for sent in test_tagged_sents:
        words = [w for w,_ in sent]
        gold  = [t for _,t in sent]
        pred  = viterbi_decode(words, tags, vocab_hmm, log_pi, log_A, log_B)

        for g, p in zip(gold, pred):
            total += 1
            if g == p:
                correct += 1

    return correct / total

acc = evaluate_hmm(test_sents[:200])  # evaluate on subset for speed
print(f"HMM POS Tagger accuracy (first 200 test sents): {acc:.4f}")


HMM POS Tagger accuracy (first 200 test sents): 0.8923


## Part 3 — N-gram Language Model (Autocomplete)

###  Goal

Build an **N-gram Language Model** for autocomplete:

- Unigram, Bigram, Trigram
- Add-k smoothing
- Compute perplexity on test set
- Autocomplete next word
- Compare unigram vs bigram vs trigram


In [21]:
# =========================
# Part 3.1 — Prepare corpus text and train/test split
# We'll use Brown corpus words for language modeling
# =========================

brown_sents = list(brown.sents()) # Convert to list for shuffling
random.seed(42)
random.shuffle(brown_sents)

split = int(0.9 * len(brown_sents))
lm_train_sents = brown_sents[:split]
lm_test_sents  = brown_sents[split:]

def preprocess_sentence(sent):
    # Lowercase + keep words only
    out=[]
    for w in sent:
        w = re.sub(r"[^A-Za-z']+", "", w)
        if w:
            out.append(w.lower())
    return out

lm_train = [preprocess_sentence(s) for s in lm_train_sents]
lm_test  = [preprocess_sentence(s) for s in lm_test_sents]

# Remove empty
lm_train = [s for s in lm_train if len(s)>0]
lm_test  = [s for s in lm_test if len(s)>0]

print("Train sentences:", len(lm_train))
print("Test sentences:", len(lm_test))
print("Example:", lm_train[0][:15])

Train sentences: 51190
Test sentences: 5691
Example: ['he', 'let', 'her', 'tell', 'him', 'all', 'about', 'the', 'church']


In [22]:
# =========================
# Part 3.2 — Build N-gram counts
# =========================

BOS = "<s>"
EOS = "</s>"
UNK = "<UNK>"

def build_ngram_counts(sentences, n:int):
    """Return n-gram counts and (n-1)-gram context counts."""
    ngram_counts = Counter()
    context_counts = Counter()
    unigram_counts = Counter()

    for sent in sentences:
        sent = [w for w in sent if w]
        # pad
        padded = [BOS]*(n-1) + sent + [EOS]
        for i in range(len(padded) - n + 1):
            ngram = tuple(padded[i:i+n])
            context = tuple(padded[i:i+n-1])
            ngram_counts[ngram] += 1
            context_counts[context] += 1
        for w in sent:
            unigram_counts[w]+=1

    vocab = set(unigram_counts.keys())
    return vocab, unigram_counts, context_counts, ngram_counts

vocab_lm, uni_counts, ctx2_counts, bi_counts = build_ngram_counts(lm_train, n=2)
_, _, ctx3_counts, tri_counts = build_ngram_counts(lm_train, n=3)

print(f"LM vocab size: {len(vocab_lm):,}")
print("Top 10 unigrams:", uni_counts.most_common(10))


LM vocab size: 45,127
Top 10 unigrams: [('the', 62986), ('of', 32765), ('and', 26002), ('to', 23572), ('a', 21078), ('in', 19236), ('that', 9500), ('is', 9115), ('was', 8861), ('he', 8592)]


In [23]:
# =========================
# Part 3.3 — Add-k smoothing probabilities
# =========================

def prob_unigram(w, uni_counts, vocab_size, k=1.0):
    return (uni_counts[w] + k) / (sum(uni_counts.values()) + k*vocab_size)

def prob_ngram(context: Tuple[str,...], word: str, ngram_counts: Counter, context_counts: Counter, vocab_size: int, k=1.0):
    """P(word | context) with add-k smoothing"""
    return (ngram_counts[context + (word,)] + k) / (context_counts[context] + k*vocab_size)

def sentence_log_prob(sent: List[str], n: int, k=1.0):
    """Compute log probability for a sentence using n-gram model."""
    sent = [w.lower() for w in sent if w]
    padded = [BOS]*(n-1) + sent + [EOS]
    V = len(vocab_lm) + 1  # plus UNK

    logp = 0.0
    for i in range(len(padded)-n+1):
        context = tuple(padded[i:i+n-1])
        w = padded[i+n-1]
        if w not in vocab_lm and w not in (BOS, EOS):
            w = UNK

        if n == 1:
            logp += math.log(prob_unigram(w, uni_counts, V, k))
        elif n == 2:
            logp += math.log(prob_ngram(context, w, bi_counts, ctx2_counts, V, k))
        else:
            logp += math.log(prob_ngram(context, w, tri_counts, ctx3_counts, V, k))
    return logp

def perplexity(sentences: List[List[str]], n: int, k=1.0, max_sentences: int = 2000):
    """Perplexity on a subset for speed."""
    sents = sentences[:max_sentences]
    N = 0
    total_logp = 0.0
    for sent in sents:
        # number of predicted tokens ~ len(sent)+1 for EOS
        N += (len(sent) + 1)
        total_logp += sentence_log_prob(sent, n=n, k=k)
    return math.exp(-total_logp / N)

for n in [1,2,3]:
    pp = perplexity(lm_test, n=n, k=1.0, max_sentences=2000)
    print(f"Perplexity n={n}: {pp:.2f}")


Perplexity n=1: 2095.69
Perplexity n=2: 5556.06
Perplexity n=3: 22624.34


In [24]:
# =========================
# Part 3.4 — Autocomplete next word
# =========================

def top_next_words(prefix: List[str], n: int = 3, k: float = 1.0, top_k: int = 5):
    """Return top_k next-word predictions given prefix."""
    prefix = [w.lower() for w in prefix if w]
    V = len(vocab_lm) + 1

    if n == 1:
        # just most frequent unigrams
        candidates = []
        for w, c in uni_counts.items():
            p = prob_unigram(w, uni_counts, V, k)
            candidates.append((p, w))
        candidates.sort(reverse=True)
        return candidates[:top_k]

    # n>=2, build context
    context = [BOS]*(n-1)
    context = context + prefix
    context = tuple(context[-(n-1):])

    candidates = []
    for w in list(vocab_lm)[:50000]:  # limit for speed
        if n == 2:
            p = prob_ngram(context, w, bi_counts, ctx2_counts, V, k)
        else:
            p = prob_ngram(context, w, tri_counts, ctx3_counts, V, k)
        candidates.append((p, w))

    candidates.sort(reverse=True)
    return candidates[:top_k]

# Demo autocomplete
prefix = "the united".split()
print("Prefix:", prefix)
print("Bigram:", top_next_words(prefix, n=2))
print("Trigram:", top_next_words(prefix, n=3))


Prefix: ['the', 'united']
Bigram: [(0.007769291545957335, 'states'), (0.0009656746554297253, 'nations'), (0.00010973575629883242, "states'"), (8.778860503906593e-05, 'to'), (8.778860503906593e-05, 'and')]
Trigram: [(0.006640427450031883, 'states'), (0.0008135623034807274, 'nations'), (0.00010994085182171991, "states'"), (4.397634072868797e-05, 'steel'), (4.397634072868797e-05, 'state')]
